In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
print("Current folder contents:")
print(os.listdir())

Current folder contents:
['.ipynb_checkpoints', 'hackathon.ipynb', 'orders.csv', 'restaurants.sql', 'users.json']


In [3]:
orders = pd.read_csv("orders.csv")
users = pd.read_json("users.json")

print("Orders shape:", orders.shape)
print("Users shape:", users.shape)

Orders shape: (10000, 6)
Users shape: (3000, 4)


In [4]:
conn = sqlite3.connect("restaurant.db")
cursor = conn.cursor()

with open("restaurants.sql", "r", encoding="utf-8") as f:
    sql_script = f.read()

cursor.executescript(sql_script)
conn.commit()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables in DB:", cursor.fetchall())

Tables in DB: [('restaurants',)]


In [5]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
print("Restaurants shape:", restaurants.shape)
restaurants.head()

Restaurants shape: (500, 4)


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [6]:
df = orders.merge(users, on="user_id", how="left")
df = df.merge(restaurants, on="restaurant_id", how="left")

print("Final dataset shape:", df.shape)
df.head()

Final dataset shape: (10000, 12)


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [7]:
df.to_csv("final_food_delivery_dataset.csv", index=False)
print("final_food_delivery_dataset.csv saved successfully")

final_food_delivery_dataset.csv saved successfully


In [8]:
res1 = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)
print(res1)

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64


In [9]:
res2 = (
    df.groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)
print(res2)

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64


In [10]:
count_users = (
    df.groupby("user_id")["total_amount"]
    .sum()
    .gt(1000)
    .sum()
)
print("Users spending > 1000:", count_users)

Users spending > 1000: 2544


In [25]:
df["rating_range"] = pd.cut(
    df["rating"],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0],
    labels=["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"],
    include_lowest=True
)

res4 = (
    df.groupby("rating_range", observed=False)["total_amount"]
    .sum()
    .sort_values(ascending=False)
)
print(res4)

rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64


In [12]:
res5 = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)
print(res5)

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64


In [13]:
res6 = (
    df.groupby("cuisine")
    .agg(
        restaurant_count=("restaurant_name_y", "nunique"),
        total_revenue=("total_amount", "sum")
    )
    .sort_values("restaurant_count")
)
print(res6)

         restaurant_count  total_revenue
cuisine                                 
Chinese               120     1930504.65
Indian                126     1971412.58
Italian               126     2024203.80
Mexican               128     2085503.09


In [14]:
gold_percentage = round(
    (df[df["membership"] == "Gold"].shape[0] / df.shape[0]) * 100
)
print("Gold order percentage:", gold_percentage)

Gold order percentage: 50


In [15]:
res8 = (
    df.groupby("restaurant_name_y")
    .agg(
        order_count=("total_amount", "count"),
        avg_order_value=("total_amount", "mean")
    )
    .query("order_count < 20")
    .sort_values("avg_order_value", ascending=False)
)
print(res8.head(1))

                   order_count  avg_order_value
restaurant_name_y                              
Restaurant_294              13      1040.222308


In [16]:
valid_options = [
    ("Gold", "Indian"),
    ("Gold", "Italian"),
    ("Regular", "Indian"),
    ("Regular", "Chinese")
]

res9 = (
    df.groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .loc[valid_options]
    .sort_values(ascending=False)
)
print(res9)

membership  cuisine
Gold        Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
Regular     Chinese     952790.91
Name: total_amount, dtype: float64


In [26]:
df["order_date"] = pd.to_datetime(
    df["order_date"],
    format="%d-%m-%Y"
)
df["quarter"] = df["order_date"].dt.to_period("Q")

res10 = (
    df.groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)
print(res10)

quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64


In [18]:
gold_orders = df[df["membership"] == "Gold"].shape[0]
print(gold_orders)

4987


In [19]:
hyd_revenue = round(
    df[df["city"] == "Hyderabad"]["total_amount"].sum()
)
print(hyd_revenue)

1889367


In [20]:
distinct_users = df["user_id"].nunique()
print(distinct_users)

2883


In [21]:
gold_aov = round(
    df[df["membership"] == "Gold"]["total_amount"].mean(),
)
print(gold_aov)

797


In [22]:
high_rating_orders = df[df["rating"] >= 4.5].shape[0]
print(high_rating_orders)

3374


In [23]:
top_gold_city = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
    .index[0]
)

print(top_gold_city)

Chennai


In [24]:
orders_top_gold_city = df[
    (df["membership"] == "Gold") &
    (df["city"] == top_gold_city)
].shape[0]

print(orders_top_gold_city)

1337
